In [10]:
# -*- coding: utf-8 -*-
"""
Classificador Binário de Cães e Gatos usando Logic Tensor Networks (LTN)
Dataset: Microsoft Cats vs Dogs (Hugging Face)
"""

# Instalação de dependências
!pip install ltn
!pip install torch torchvision
!pip install datasets transformers

# Dataset Usado
disponivel em: https://www.microsoft.com/en-us/download/details.aspx?id=54765


In [11]:
# montar no drive e carregar as variaveis path

In [12]:
from google.colab import drive
drive.mount('/content/drive')
pathcat = '/content/drive/MyDrive/DatasetCaes&Gatos/cats'
pathdog = '/content/drive/MyDrive/DatasetCaes&Gatos/dogs'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import random
import os
from pathlib import Path

# ======= Configuração dos caminhos do dataset =======
pathcat = '/content/drive/MyDrive/DatasetCaes&Gatos/cats'
pathdog = '/content/drive/MyDrive/DatasetCaes&Gatos/dogs'

# ======= Semente para reprodutibilidade =======
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =====================================================================
# 1. DATASET E DATALOADER (MODIFICADO PARA PASTAS LOCAIS)
# =====================================================================

class LocalDogCatDataset(Dataset):
    """
    Dataset usando pastas locais 'dogs' e 'cats'
    Retorna pares (dog_img, cat_img) como no código original.
    """
    def __init__(self, dogs_dir='dogs', cats_dir='cats', transform=None, max_samples=None):
        print("📥 Carregando imagens das pastas locais...")

        # Convertendo para Path objects
        self.dogs_dir = Path(dogs_dir)
        self.cats_dir = Path(cats_dir)

        # Verificar se os diretórios existem
        if not self.dogs_dir.exists():
            raise ValueError(f"❌ Diretório de cães não encontrado: {dogs_dir}")
        if not self.cats_dir.exists():
            raise ValueError(f"❌ Diretório de gatos não encontrado: {cats_dir}")

        # Extensões de imagem suportadas
        valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp'}

        # Carregar caminhos das imagens de cães
        self.dog_images = [
            str(f) for f in self.dogs_dir.iterdir()
            if f.is_file() and f.suffix.lower() in valid_extensions
        ]

        # Carregar caminhos das imagens de gatos
        self.cat_images = [
            str(f) for f in self.cats_dir.iterdir()
            if f.is_file() and f.suffix.lower() in valid_extensions
        ]

        # Embaralhar para aleatoriedade
        random.shuffle(self.dog_images)
        random.shuffle(self.cat_images)

        # Limitar número de amostras se especificado
        if max_samples is not None and max_samples > 0:
            self.dog_images = self.dog_images[:max_samples]
            self.cat_images = self.cat_images[:max_samples]

        self.transform = transform

        print(f"✓ Cães encontrados: {len(self.dog_images)}")
        print(f"✓ Gatos encontrados: {len(self.cat_images)}")
        print(f"✓ Pares balanceados: {min(len(self.dog_images), len(self.cat_images))}")

        if len(self.dog_images) == 0 or len(self.cat_images) == 0:
            raise ValueError("❌ Erro: Não foram encontradas imagens suficientes de cães ou gatos!")

    def __len__(self):
        return min(len(self.dog_images), len(self.cat_images))

    def __getitem__(self, idx):
        # Carregar imagens com retry automático em caso de erro
        max_retries = 10

        for attempt in range(max_retries):
            try:
                # Ajustar índice se necessário
                dog_idx = (idx + attempt) % len(self.dog_images)
                cat_idx = (idx + attempt) % len(self.cat_images)

                dog_path = self.dog_images[dog_idx]
                cat_path = self.cat_images[cat_idx]

                # Tentar carregar imagem de cão
                dog_img = Image.open(dog_path).convert('RGB')
                # Verificar se a imagem é válida tentando carregá-la
                dog_img.load()

                # Tentar carregar imagem de gato
                cat_img = Image.open(cat_path).convert('RGB')
                # Verificar se a imagem é válida tentando carregá-la
                cat_img.load()

                # Aplicar transformações
                if self.transform:
                    dog_img = self.transform(dog_img)
                    cat_img = self.transform(cat_img)

                return dog_img, cat_img

            except Exception as e:
                if attempt == 0:  # Só printa no primeiro erro
                    print("rodando...")
                continue

        # Se todas as tentativas falharem, retorna imagens em branco
        print(f"❌ Não foi possível carregar imagens válidas após {max_retries} tentativas")
        dog_img = torch.randn(3, 96, 96)  # imagem aleatória
        cat_img = torch.randn(3, 96, 96)
        return dog_img, cat_img


def get_transforms(image_size=128, augmentation=True):
    """Define transformações para as imagens (um pouco menos agressivas)"""
    if augmentation:
        return transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(8),
            transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


def create_dataloader(dogs_dir='dogs', cats_dir='cats', batch_size=16,
                      image_size=128, num_workers=0, augmentation=True, max_samples=None):
    """
    Cria DataLoader a partir de pastas locais

    Args:
        dogs_dir: caminho para pasta com imagens de cães
        cats_dir: caminho para pasta com imagens de gatos
        batch_size: tamanho do batch
        image_size: tamanho das imagens (redimensionadas)
        num_workers: número de workers para carregamento
        augmentation: aplicar data augmentation
        max_samples: número máximo de amostras por classe
    """
    transform = get_transforms(image_size, augmentation)
    dataset = LocalDogCatDataset(
        dogs_dir=dogs_dir,
        cats_dir=cats_dir,
        transform=transform,
        max_samples=max_samples
    )

    if len(dataset) == 0:
        raise ValueError("Dataset está vazio! Verifique se as imagens foram carregadas corretamente.")

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )
    print(f"✓ DataLoader criado com {len(dataloader)} batches\n")
    return dataloader


# =====================================================================
# 2. MODELO CNN (REMOVIDO SIGMOID FINAL)
# =====================================================================

class CNNModel(nn.Module):
    """
    CNN sem Sigmoid final — vamos usar BCEWithLogitsLoss nos logits.
    """
    def __init__(self, input_channels=3, num_classes=1):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)  # logits (sem Sigmoid)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.fc(x)
        return x  # logits


# =====================================================================
# 3. TREINAMENTO COM LTN (AJUSTADO)
# =====================================================================

def train_ltn_classifier(dogs_dir='dogs', cats_dir='cats', n_epochs=5,
                         batch_size=64, image_size=96, learning_rate=0.001,
                         max_samples=500, device=None):
    """
    Treinamento com BCEWithLogits + LTN fuzzy. Épocas padrão reduzidas para 10.
    """
    device = device or (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
    print(f"🖥️  Usando device: {device}\n")

    # DataLoader
    print("=" * 60)
    print("ETAPA 1: Carregando Dataset")
    print("=" * 60)
    train_dataloader = create_dataloader(
        dogs_dir=dogs_dir,
        cats_dir=cats_dir,
        batch_size=batch_size,
        image_size=image_size,
        num_workers=0,
        augmentation=True,
        max_samples=max_samples
    )

    # Modelo
    print("=" * 60)
    print("ETAPA 2: Criando Modelo CNN")
    print("=" * 60)
    model = CNNModel(input_channels=3, num_classes=1)
    model = model.to(device)
    print(f"✓ Modelo criado com {sum(p.numel() for p in model.parameters())} parâmetros\n")

    # Otimizador e scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # BCEWithLogitsLoss com pos_weight > 1 para priorizar cães (classe positiva)
    pos_weight = torch.tensor([1.5]).to(device)
    bce_loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_loss = float('inf')

    print("=" * 60)
    print("ETAPA 3: Iniciando Treinamento")
    print("=" * 60)

    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0.0
        correct_dogs = 0
        correct_cats = 0
        total_samples = 0

        for i, (dog_imgs, cat_imgs) in enumerate(train_dataloader):
            dog_imgs = dog_imgs.to(device)
            cat_imgs = cat_imgs.to(device)

            # Concatena cães + gatos para formar um batch único
            inputs = torch.cat([dog_imgs, cat_imgs], dim=0)
            labels = torch.cat([
                torch.ones(dog_imgs.size(0), 1, device=device),
                torch.zeros(cat_imgs.size(0), 1, device=device)
            ], dim=0)

            optimizer.zero_grad()

            logits = model(inputs)

            # LTN fuzzy
            probs = torch.sigmoid(logits)
            dogs_probs = probs[:dog_imgs.size(0)]
            cats_probs = probs[dog_imgs.size(0):]
            phi1 = torch.mean(dogs_probs)
            phi2 = torch.mean(1.0 - cats_probs)
            sat_agg = (phi1 + phi2) / 2.0
            ltn_loss = 1.0 - sat_agg

            # BCEWithLogitsLoss
            bce_total = bce_loss_fn(logits, labels)

            # Combinar losses
            loss = 0.4 * ltn_loss + 0.6 * bce_total

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()

            # Métricas de acurácia
            with torch.no_grad():
                preds = (probs > 0.5).float()
                correct_dogs += preds[:dog_imgs.size(0)].sum().item()
                correct_cats += (1 - preds[dog_imgs.size(0):]).sum().item()
                total_samples += inputs.size(0)

            if (i + 1) % 5 == 0 or (i + 1) == len(train_dataloader):
                batch_acc = (correct_dogs + correct_cats) / total_samples * 100
                print(f"Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{len(train_dataloader)}], "
                      f"Loss: {loss.item():.4f}, Acc(batch): {batch_acc:.2f}%")

        epoch_loss = epoch_loss / len(train_dataloader)
        epoch_acc = (correct_dogs + correct_cats) / total_samples * 100

        scheduler.step(epoch_loss)

        print(f"\n{'='*60}")
        print(f"Epoch [{epoch+1}/{n_epochs}] Completado")
        print(f"Loss Médio: {epoch_loss:.4f}")
        print(f"Acurácia: {epoch_acc:.2f}%")
        print(f"Cães corretos (aprx): {int(correct_dogs)}/{total_samples//2}")
        print(f"Gatos corretos (aprx): {int(correct_cats)}/{total_samples//2}")
        print(f"{'='*60}\n")

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            print(f"💾 Novo melhor modelo (na memória). Loss: {best_loss:.4f}\n")

    print("🎉 Treinamento concluído!\n")
    return model, train_dataloader


# =====================================================================
# 4. AVALIAÇÃO E TESTES
# =====================================================================

def evaluate_model(model, dataloader, device, num_samples=5):
    model.eval()
    print("=" * 60)
    print("AVALIAÇÃO DO MODELO")
    print("=" * 60)

    with torch.no_grad():
        dog_imgs, cat_imgs = next(iter(dataloader))
        dog_imgs = dog_imgs[:num_samples].to(device)
        cat_imgs = cat_imgs[:num_samples].to(device)

        inputs = torch.cat([dog_imgs, cat_imgs], dim=0)
        logits = model(inputs)
        probs = torch.sigmoid(logits)

        dog_probs = probs[:num_samples]
        cat_probs = probs[num_samples:]

        print(f"\nPredições para {num_samples} imagens de CÃES:")
        for i, p in enumerate(dog_probs):
            print(f"  Imagem {i+1}: {p.item():.4f} (esperado: ~1.0)")

        print(f"\nPredições para {num_samples} imagens de GATOS:")
        for i, p in enumerate(cat_probs):
            print(f"  Imagem {i+1}: {p.item():.4f} (esperado: ~0.0)")

        dog_correct = (dog_probs > 0.5).sum().item()
        cat_correct = (cat_probs <= 0.5).sum().item()
        accuracy = (dog_correct + cat_correct) / (2 * num_samples) * 100

        print(f"\n✓ Acurácia nas amostras: {accuracy:.2f}%")
        print("=" * 60 + "\n")


# =====================================================================
# 5. EXECUÇÃO PRINCIPAL
# =====================================================================

if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("CLASSIFICADOR BINÁRIO: CÃES vs GATOS (DATASET LOCAL)")
    print("Usando BCEWithLogits + Lógica fuzzy (LTN simplificada)")
    print("=" * 60 + "\n")

    print(f"📂 Pasta de cães: {pathdog}")
    print(f"📂 Pasta de gatos: {pathcat}\n")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Usando as variáveis pathdog e pathcat definidas no início
    model, dataloader = train_ltn_classifier(
        dogs_dir=pathdog,      # <- Usando a variável pathdog
        cats_dir=pathcat,      # <- Usando a variável pathcat
        n_epochs=5,            # <- Reduzido para 5 épocas
        batch_size=64,         # <- Aumentado para processar mais imagens por vez
        image_size=96,         # <- Reduzido para 96x96 (mais rápido)
        learning_rate=0.001,   # <- Learning rate um pouco maior
        max_samples=500,       # <- APENAS 500 IMAGENS DE CADA (muito mais rápido!)
        device=device
    )

    # Avaliação rápida
    evaluate_model(model, dataloader, device, num_samples=5)


CLASSIFICADOR BINÁRIO: CÃES vs GATOS (DATASET LOCAL)
Usando BCEWithLogits + Lógica fuzzy (LTN simplificada)

📂 Pasta de cães: /content/drive/MyDrive/DatasetCaes&Gatos/dogs
📂 Pasta de gatos: /content/drive/MyDrive/DatasetCaes&Gatos/cats

🖥️  Usando device: cuda

ETAPA 1: Carregando Dataset
📥 Carregando imagens das pastas locais...
✓ Cães encontrados: 500
✓ Gatos encontrados: 500
✓ Pares balanceados: 500
✓ DataLoader criado com 8 batches

ETAPA 2: Criando Modelo CNN
✓ Modelo criado com 10980673 parâmetros

ETAPA 3: Iniciando Treinamento


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch [1/5], Step [5/8], Loss: 0.7140, Acc(batch): 50.00%
rodando...
Epoch [1/5], Step [8/8], Loss: 0.7558, Acc(batch): 50.30%

Epoch [1/5] Completado
Loss Médio: 1.0817
Acurácia: 50.30%
Cães corretos (aprx): 342/500
Gatos corretos (aprx): 161/500

💾 Novo melhor modelo (na memória). Loss: 1.0817

Epoch [2/5], Step [5/8], Loss: 0.7748, Acc(batch): 50.94%
rodando...
Epoch [2/5], Step [8/8], Loss: 0.7303, Acc(batch): 50.90%

Epoch [2/5] Completado
Loss Médio: 0.7499
Acurácia: 50.90%
Cães corretos (aprx): 354/500
Gatos corretos (aprx): 155/500

💾 Novo melhor modelo (na memória). Loss: 0.7499

Epoch [3/5], Step [5/8], Loss: 0.6840, Acc(batch): 51.88%
rodando...
Epoch [3/5], Step [8/8], Loss: 0.6531, Acc(batch): 52.70%

Epoch [3/5] Completado
Loss Médio: 0.6912
Acurácia: 52.70%
Cães corretos (aprx): 385/500
Gatos corretos (aprx): 142/500

💾 Novo melhor modelo (na memória). Loss: 0.6912

rodando...
Epoch [4/5], Step [5/8], Loss: 0.6478, Acc(batch): 53.59%
Epoch [4/5], Step [8/8], Loss: 0.6511